In [1]:
import re
import time

import pandas as pd
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

import pickle

from heapq import nlargest

import tensorflow as tf
import tensorflow_hub as hub

In [2]:
# Загрузим датасет.

df = pd.read_csv('train.csv')

In [3]:
# Посмотрим на данные.

df.head()

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0


In [4]:
df.shape

(497819, 4)

In [5]:
# Загрузим предобученный universal-sentence-encoder

use = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

In [6]:
# Сделаем списки названий компаний.

df_brand_names = pd.DataFrame(pd.concat([df.name_1, df.name_2], axis=0).drop_duplicates(), columns=['name'])
brand_names = df_brand_names.name.tolist()

In [7]:
# Получим эмбеддинги из universal-sentence-encoder.

brand_embeddings = use(brand_names)

In [8]:
# Функция для нахождения топ 10 похожих на исходное названий.

def find_similar_name(sentence, embeddings):
    
    sentence_embedding = use([sentence])
    similarity = cosine_similarity(sentence_embedding, embeddings)
    top = sorted(enumerate(*similarity), key=lambda x: x[1], reverse=True)
    similar_names = [brand_names[x[0]] for x in top[:10]]
    for name in similar_names:
        print(name)
    

In [9]:
# Время начала обработки поиска. 
start_time = time.time()

# Найдём топ 10 похожих названий для компании Mediterranean Textile Co, которая была в датасете.

print(brand_names[80])
print()
find_similar_name(brand_names[80], brand_embeddings)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Mediterranean Textile Co.

Mediterranean Textile Co.
Zamoung Textile Co.
Torreh Textile Co.
Kavir Semnan Textile Co.
Ardakan Textile Co.
Ferdows Textile Co.
Hanam Textile Co.
Mediterranean Textile
Razi Abhar Textile Co.
Alroubaia Textile Co Fourtex

Время обработки поиска =  0.377


In [10]:
# Вдими саму компанию и названия с такими же словами.

In [11]:
# Время начала обработки поиска. 
start_time = time.time()

# Теперь получим эмбеддинг и найдём похожие названия для строки, которой не было в датасете.

my_text_test_1 = 'Darth Vader is a great Sith Lord'
print(my_text_test_1)
print()

find_similar_name(my_text_test_1, brand_embeddings)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Darth Vader is a great Sith Lord

Hussein Hussein Hezam Mohmmed
Sonic Fluid Power Systems (Pvt.)
Atat Tasimacilik Cozumleri A.S.
Abdullah Hussain Ali
The Cato Corporation
Ashoka Inc.
Sound Merch Bmg (Neutral)
Hyundai Power Transformers Usa
Sika S.A. Chile
Kia Chile S.A.

Время обработки поиска =  0.1


In [12]:
# Интересно, что к Дарту Вейдеру модель вывела названия, где фигурировало слово Power. Аж два названия.
# "Atat" возможно из-за похожести на AT-AT (Имперский шагоход). 
# И "Ashoka Inc." из-за похожести  "Ashoka" на "Ahsoka". 
# Потому что Ahsoka Tano это ученица Вейдера, времён, когда его еще Энакином звали.

In [13]:
# Время начала обработки поиска. 
start_time = time.time()


# Теперь получим эмбеддинг и найдём похожие названия для строки, похожей, на то, что было в датасете.
# В датасете было название "Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V."
# Посмотрим, что на него выдаст модель.

my_text_test_2 = 'Well Aero System Lab Industrial Craft'
print(my_text_test_2)
print()

find_similar_name(my_text_test_2, brand_embeddings)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Well Aero System Lab Industrial Craft

Golden Wheel Industrial Mfg. Corp
Titan Rubber Industrial Mfg. Corp
Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
Mova Industrial S.A.C.
Newpro Industrial Mfg. Corp
Aero Club
Erbus Industrial Eireli
Aero Traders
Distribuidora Industrial Avanzada Sa De Cv
Pegsa Industrial Sac

Время обработки поиска =  0.123


In [14]:
# Видим, что нашлись названия компаний с похожими словами:  "Industrial", "Aero", "Aerospace", "Laboratory".  
# Время обработки поиска 0.1-0.3 секунды.